In [ ]:
from langchain.chat_models import ChatOpenAI, ChatAnthropic

chat = ChatOpenAI()
# chat2 = ChatAnthropic() 다른 모델도 이렇게 사용할 수 있음

# chat = ChatOpenAI(
#     max_tokens=
#     temperature= 
#       ...
# )
# 위와 같은 설정으로 모델을 어떻게 사용할 것인지 정할 수 있다.

b = chat.predict("How many people in South Korea?")

b

'As of July 2021, the population of South Korea is estimated to be around 51 million people.'

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# HumanMessage : 우리가 보내는 것. 사용자의 메세지
# AIMessage : AI에 의해 보내지는 것 
# SystemMessage : 우리가 LLM에 설정들을 제공하기 위한 Message. 프롬프트

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian."
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)
# 위와 같은 방식으로 더 복잡한 대화를 만들 수 있다.
# 일종의 설정을 AI에게 넘겨줬고, 마치 AI가 답변을 한 것으로 보여준다.
# 위의 결과값 :
# AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono qui per aiutarti con le tue domande di geografia. Avete altre domande?')


AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono qui per aiutarti con le tue domande di geografia. Avete altre domande?')

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# ChatPromptTemplate : template을 message로부터 만든다.
# PromptTemplate : 그냥 String을 이용해서 template을 만든다.

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)

prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 9,600 miles (15,450 kilometers) when measured in a straight line.'

In [ ]:
# 위의 prompt messages를 사용해서 위의 방식을 조금 더 간편하게 바꿀 수 있다.
template = ChatPromptTemplate.from_messages([
    # SystemMessage(
    #     content="You are a geography expert. And you only reply in Italian."
    # ),
    # AIMessage(content="Ciao, mi chiamo Paolo!"),
    # HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)
# 결과값: AIMessage(content='Γεια σας, το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')


AIMessage(content='Γεια σας, το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')